# Synopsis

# Configuration

In [1]:
db_name = 'persuasion.db'
OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']

# Libraries

In [2]:
import sqlite3
import pandas as pd
import numpy as np

# Process

In [3]:
with sqlite3.connect(db_name) as db:
    K = pd.read_sql('SELECT * FROM token', db, index_col=OHCO)
    V = pd.read_sql('SELECT * FROM vocab', db, index_col='term_id')

## Create DTM


### Create word mask

Let's filter out stopwords -- another hyperparameter. 

In [4]:
WORDS = (K.punc == 0) & (K.num == 0) & K.term_id.isin(V[V.stop==0].index)

### Extrct BOW from tokens

To extract a bag-of-words model from our tokens table, we apply a simple `groupby()` operation. Note that we can drop in our hyperparameters easily -- CHAPS and 'term_id' and be replaced. We can easily write a function to simplify this process and make it more configurable. 


In [5]:
BOW = K[WORDS].groupby(OHCO[:1]+['term_id'])['term_id'].count()

### Convert BOW to DTM

In [6]:
DTM = BOW.unstack().fillna(0)

## Compute Term Frequencies and Weights

### Compute TF

In [7]:
alpha = .000001 # We introduce an arbitrary smoothing value
alpha_sum = alpha * V.shape[0]
TF = DTM.apply(lambda x: (x + alpha) / (x.sum() + alpha_sum), axis=1)

### Compute TFIDF

In [8]:
N_docs = DTM.shape[0]
V['df'] = DTM[DTM > 0].count()
TFIDF = TF * np.log2(N_docs / V[V.stop==0]['df'])

### Compute TFTH (Experiment)

In [9]:
THM = -(TF * np.log2(TF))
TFTH = TF.apply(lambda x: x * THM.sum(), 1)

### Add stats to V

In [10]:
V['tf_sum'] = TF.sum()
V['tf_mean'] = TF.mean()
V['tf_max'] = TF.max()
V['tfidf_sum'] = TFIDF.sum()
V['tfidf_mean'] = TFIDF.mean()
V['tfidf_max'] = TFIDF.max()
V['tfth_sum'] = TFTH.sum()
V['tfth_mean'] = TFTH.mean()
V['tfth_max'] = TFTH.max()
V['th_sum'] = THM.sum()
V['th_mean'] = THM.mean()
V['th_max'] = THM.max()

## Create Docs table

In [11]:
D = DTM.sum(1).astype('int').to_frame().rename(columns={0:'term_count'})
D['tf'] = D.term_count / D.term_count.sum()

## Get all doc pairs

In [12]:
chap_ids = D.index.tolist()
pairs = [(i,j) for i in chap_ids for j in chap_ids if j > i]
P = pd.DataFrame(pairs).reset_index(drop=True).set_index([0,1])
P.index.names = ['doc_x','doc_y']

## Compute Euclidean distance

In [13]:
def euclidean(row):
    D1 = TFIDF.loc[row.name[0]]
    D2 = TFIDF.loc[row.name[1]]
    x = (D1 - D2)**2
    y = x.sum() 
    z = np.sqrt(y)
    return z

In [14]:
P['euclidean'] = 0
P['euclidean'] = P.apply(euclidean, 1)

## Compute Cosine similarity

In [1]:
def cosine(row):
    D1 = TFIDF.loc[row.name[0]]
    D2 = TFIDF.loc[row.name[1]]
    x = D1 * D2
    y = x.sum()
    a = np.sqrt((D1**2).sum())
    b = np.sqrt((D2**2).sum())
    c = a * b
    z = y / c
    return z

In [16]:
P['cosine'] = P.apply(cosine, 1)

# Save data

In [17]:
with sqlite3.connect(db_name) as db:
    V.to_sql('vocab', db, if_exists='replace', index=True)
    K.to_sql('token', db, if_exists='replace', index=True)
    D.to_sql('doc', db, if_exists='replace', index=True)
    P.to_sql('docpair', db, if_exists='replace', index=True)
#     BOW.to_frame().rename(columns={'term_id':'n'}).to_sql('bow', db, if_exists='replace', index=True)
    TFIDF.stack().to_frame().rename(columns={0:'term_weight'})\
        .to_sql('dtm_tfidf', db, if_exists='replace', index=True)

In [18]:
# END